# Solveur de Jacobi : Modèles de programmation Multi-GPU
Ce notebook présente 12 versions progressives d’un solveur de Jacobi 2D. Chaque section explique le modèle ou l’optimisation, compile la version, exécute et collecte les métriques Nsight.

```markdown
## Modules Spack à charger

Avant de compiler ou d’exécuter les différentes étapes, il est recommandé de charger les modules nécessaires via Spack. Par exemple :

```bash
spack load cuda@12.6
spack load nvidia-nsight-systems@2024.6.1
spack load nvhpc@24.11 
spack load cudnn@9.2.0.82-12
```

Adaptez la version de chaque module selon la configuration de votre cluster.
```

## etape1_cpu
**Description :** Solveur Jacobi CPU de base : implémentation mono-thread. Utile pour valider la correction et les petites tailles de problème ; met en évidence la limite de calcul CPU.

**Intérêt :** Baseline : évalue la limite CPU pour établir une référence.

In [39]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape1_cpu
make clean all

rm -f main
gcc -O2 -o main main.c


In [ ]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape1_cpu
nsys profile --stats=true --force-overwrite true -o main ./main

Error while terminating subprocess (pid=2550785): 


Generating '/tmp/nsys-report-709b.qdstrm'
[1/8] [========================100%] main.nsys-rep
[2/8] [========================100%] main.sqlite


SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain NV Tools Extension (NVTX) data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain OS Runtime trace data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA trace data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA kernel data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain GPU memory data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain GPU memory data.


[3/8] Executing 'nvtx_sum' stats report
[4/8] Executing 'osrt_sum' stats report
[5/8] Executing 'cuda_api_sum' stats report
[6/8] Executing 'cuda_gpu_kern_sum' stats report
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report
Generated:
    /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.nsys-rep
    /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite


## etape2_cpu_gpu
**Description :** 1 CPU + 1 GPU + 1 stream : le CPU pilote le GPU via un unique stream CUDA. Le calcul Jacobi est entièrement délégué au GPU, le CPU ne fait que l’orchestration.

**Intérêt :** Met en évidence l'écart de performance CPU vs GPU lorsque la grille est suffisamment grande, dans un contexte réaliste d’utilisation d’un seul GPU et d’un seul stream.

### Compilation et exécution (1 CPU + 1 GPU + 1 stream)

In [54]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape2_cpu_gpu
make clean all

rm -f app
nvcc -O2 -o app main.cu kernel.cu


In [42]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape2_cpu_gpu
nsys profile -t cuda --stats=true --force-overwrite true -o main ./app

Terminé etape2_cpu_gpu (1CPU + 1GPU + 1stream)
GPU time: 48.874808 seconds
Generating '/tmp/nsys-report-a648.qdstrm'
[1/6] [========================100%] main.nsys-rep
[2/6] [========================100%] main.sqlite
[3/6] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)       Med (ns)     Min (ns)    Max (ns)     StdDev (ns)            Name         
 --------  ---------------  ---------  -------------  -------------  ---------  -----------  -------------  ----------------------
     88.2      42956795360          3  14318931786.7  20899479424.0   46286208  22011029728  12372962251.6  cudaMemcpyAsync       
     11.4       5554665760       1001      5549116.6      5513056.0       1824     10656032       460412.7  cudaStreamSynchronize 
      0.3        155750016          1    155750016.0    155750016.0  155750016    155750016            0.0  cudaStreamCreate      
      0.0         23931008          2     11965504.0     11965504.0   11949984     

## etape3_mpi_gpus
**Description :** MPI + GPUs : domaine réparti sur plusieurs rangs MPI, chacun pilotant un GPU. Illustrations des défis de mise à l'échelle multi-nœuds et des communications inter-rangs.

**Intérêt :** Test de montée en charge inter-nœuds et coût MPI sur cluster multi-GPU.

**Step :** 
- Initialiser MPI, déterminer le rang et le nombre de processus.
- Associer chaque rang à un GPU différent.
- Diviser la grille entre les rangs (découpage 1D vertical).
- Gérer les échanges d’halos entre rangs voisins avec MPI_Sendrecv.
- Synchroniser les échanges à chaque itération.
- Nettoyer MPI à la fin.

In [2]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
cd etape3_mpi_gpus
make clean all

rm -f jacobi
/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/bin/nvcc -O3 -std=c++14 -lcudart -Xcompiler "-fopenmp" -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include main.cpp kernel.cu -o jacobi -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/lib64 -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib -lmpi -lnccl -lstdc++ \
	-Xlinker --no-as-needed


In [8]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
cd etape3_mpi_gpus
nsys profile -t mpi,cuda --stats=true --force-overwrite true -o main mpirun -np 2 ./jacobi 1000 4096 4096 1

Done: 1000 iters in 7.20935s, norm=0.309572
Generating '/tmp/nsys-report-54f1.qdstrm'
[1/7] [========================100%] main.nsys-rep
[2/7] [========================100%] main.sqlite


SKIPPED: No data available.


[3/7] Executing 'nvtx_sum' stats report
[4/7] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)           Name         
 --------  ---------------  ---------  --------  --------  --------  --------  -----------  ----------------------
     97.4        822638144      10004   82230.9   12288.0      3328    432224     142690.9  cudaMemcpy            
      2.3         19455296       2000    9727.6    9472.0      7232    361248      10730.9  cudaLaunchKernel      
      0.1          1134080          4  283520.0  287232.0    164512    395104     125392.3  cudaFree              
      0.1           776288          4  194072.0  183776.0    153568    255168      45801.7  cudaMalloc            
      0.0           151360        826     183.2      96.0        32      3424        236.9  cuGetProcAddress_v2   
      0.0            75744          4   18936.0   14864.0      5248     40768      17099.7  cudaMemset        

### À partir de quelle taille de matrice le recouvrement communication/calcul devient-il rentable ?

Le recouvrement (overlap) communication/calcul devient généralement rentable lorsque :
- Le temps de communication (MPI + transferts host/device) devient significatif devant le temps de calcul local.
- La partie du calcul qui peut être effectuée pendant la communication (hors bords) est suffisamment grande pour masquer la latence réseau.

Pour une grille Jacobi 2D, la taille critique dépend :
- De la bande passante et latence réseau,
- Du nombre de rangs MPI,
- De la rapidité des transferts CUDA Host/Device,
- De la puissance du GPU.

**Sur la plupart des clusters modernes, le recouvrement commence à être rentable pour des matrices de l’ordre de 16k×16k à 32k×32k (voire plus),** surtout si le nombre de rangs est élevé (≥4) et que la communication devient un vrai goulot d’étranglement.

**Pour une matrice 8k×8k,** le calcul local reste souvent dominant, donc le surcoût du overlap (copies, synchronisations) peut masquer le gain.  
**Essayez avec 16k×16k ou 32k×32k** pour voir un bénéfice, surtout si vous augmentez le nombre de rangs MPI (et donc la proportion de communication).

**Résumé :**  
- < 8k×8k : overlap rarement utile  
- 16k×16k : commence à être intéressant  
- 32k×32k et + : overlap souvent rentable, surtout sur cluster multi-nœuds

## etape4_mpi_overlap
**Description :** MPI + recouvrement : recouvrements des échanges d’halo non-bloquants avec le calcul Jacobi local. Réduit l'impact de la latence réseau.

**Intérêt :** Cache la latence réseau en recouvrant communication et calcul local.

In [13]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
cd etape4_mpi_overlap
make clean all

rm -f jacobi
/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/bin/nvcc -O3 -std=c++14 -lcudart -Xcompiler "-fopenmp" -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include main.cpp kernel.cu -o jacobi -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/lib64 -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib -lmpi -lnccl -lstdc++ \
	-Xlinker --no-as-needed


In [14]:
%%bash 
cd etape4_mpi_overlap
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
nsys profile -t mpi,cuda --stats=true --force-overwrite true -o main mpirun -np 2 ./jacobi 1000 4096 4096 1

Rang 1: ny_local=2047 offset=2048
nx=4096 ny=4096 size=2
Rang 0: ny_local=2047 offset=1
Overlap: 1000 iters en 7.15792 s, norm=0.309412
Generating '/tmp/nsys-report-bcfa.qdstrm'
[1/7] [========================100%] main.nsys-rep
[2/7] [========================100%] main.sqlite


SKIPPED: No data available.


[3/7] Executing 'nvtx_sum' stats report
[4/7] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)              Name             
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  -----------------------------
     80.5        733237856       2004    365887.2    401344.0    100736    536160      41446.7  cudaMemcpy                   
     11.0        100081184          2  50040592.0  50040592.0  43052736  57028448    9882320.7  cuIpcOpenMemHandle_v2        
      3.1         28382528       2000     14191.3     13376.0     10464    573152      15836.1  cudaLaunchKernel             
      1.6         14339200       2000      7169.6      7008.0      5504     37824       1874.4  cuMemcpyDtoDAsync_v2         
      1.5         13545952         32    423311.0      2832.0      1696  12827424    2263803.8  cuStreamCreate               
      1.4         12326848      2

## etape5_nccl
**Description :** NCCL : utilisation de la NVIDIA Collective Communications Library pour les échanges GPU à GPU. Montre les gains via NVLink ou PCIe haute bande passante.

**Intérêt :** Exploite automatiquement le topologie NVLink/PCIe pour des échanges GPU efficaces.

### Introduction à NCCL

NCCL (NVIDIA Collective Communications Library) permet des communications collectives efficaces entre plusieurs GPU, en exploitant la topologie matérielle (NVLink, PCIe).  
Dans un contexte Jacobi multi-GPU, NCCL peut être utilisé pour échanger les halos entre GPUs sans repasser par le CPU.

**Exemple minimal d'utilisation de NCCL pour un échange entre deux GPU :**

In [53]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
cd etape5_nccl
make clean all

rm -f *.o jacobi_mpi_nccl
/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/bin/nvcc -O3 -std=c++14 -lcudart -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include -dc kernel.cu -o kernel.o
/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/bin/nvcc -O3 -std=c++14 -lcudart -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include main.cpp kernel.o -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/lib64 -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib -L/apps/2025/manual_insta

In [54]:
%%bash 
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
cd etape5_nccl
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
#nsys profile -t cuda --stats=true --force-overwrite true -o main 
NCCL_DEBUG=WARN mpirun -np 2 ./jacobi_mpi_nccl -niter 1000 -nx 4096 -ny 4096 -nccheck 1

NCCL version 2.19.3+cuda12.3
Single GPU jacobi relaxation: 1000 iterations on 4096 x 4096 mesh with norm check every 1 iterations
    0, 15.998031
  100, 0.448893
  200, 0.267747
  300, 0.197740
  400, 0.159443
  500, 0.134900
  600, 0.117686
  700, 0.104841
  800, 0.094847
  900, 0.086838
Jacobi relaxation: 1000 iterations on 4096 x 4096 mesh with norm check every 1 iterations
    0, 15.998049
  100, 0.448911
  200, 0.267771
  300, 0.197757
  400, 0.159458
  500, 0.134914
  600, 0.117695
  700, 0.104854
  800, 0.094863
  900, 0.086849
Num GPUs: 2.
4096x4096: 1 GPU:  29.4556 s, 2 GPUs:  14.7631 s, speedup:     2.00, efficiency:    99.76 


## etape6_nccl_overlap
**Description :** NCCL + recouvrement : superposition des collectifs NCCL avec le calcul sur GPU, cachant le coût de communication.

**Intérêt :** Essentiel à forte densité GPU pour maintenir les cœurs occupés.

In [58]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
cd etape6_nccl_overlap
make clean all

rm -f *.o jacobi_mpi_nccl_overlap
/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/bin/nvcc -O3 -std=c++14 -lcudart -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include -dc kernel.cu -o kernel.o
/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/bin/nvcc -O3 -std=c++14 -lcudart -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include main.cpp kernel.o -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/lib64 -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib -L/apps/2025/manu

In [59]:
%%bash 
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
cd etape6_nccl_overlap
#nsys profile -t mpi,cuda --stats=true --force-overwrite true -o main 
mpirun -np 2 ./jacobi_mpi_nccl_overlap -niter 1000 -nx 4096 -ny 4096 -nccheck 1

Single GPU jacobi relaxation: 1000 iterations on 4096 x 4096 mesh with norm check every 1 iterations
    0, 15.998031
  100, 0.448893
  200, 0.267747
  300, 0.197740
  400, 0.159443
  500, 0.134900
  600, 0.117686
  700, 0.104841
  800, 0.094847
  900, 0.086838
Jacobi relaxation: 1000 iterations on 4096 x 4096 mesh with norm check every 1 iterations
    0, 15.998048
  100, 0.448911
  200, 0.267771
  300, 0.197757
  400, 0.159458
  500, 0.134914
  600, 0.117695
  700, 0.104854
  800, 0.094863
  900, 0.086849
Num GPUs: 2.
4096x4096: 1 GPU:  29.4558 s, 2 GPUs:  15.2023 s, speedup:     1.94, efficiency:    96.88 


## etape7_nccl_graphs
**Description :** NCCL + CUDA Graphs : capture et relecture des séquences Jacobi/échange pour réduire le surcoût des lancements.

**Intérêt :** Réduit l’overhead de lancement grâce aux CUDA Graphs.

In [ ]:
%%bash 
cd etape7_nccl_graphs
make all

In [ ]:
%%bash 
cd etape7_nccl_graphs
nv-nsight-cu-cli --csv --report-file rapport_etape7_nccl_graphs.csv ./main
cat rapport_etape7_nccl_graphs.csv

## etape8_nvshmem
**Description :** NVSHMEM : modèle PGAS à accès mémoire unilatéral GPU, simplifiant les mises à jour d’halo.

**Intérêt :** Simplifie les échanges via modèle PGAS unilatéral.

In [ ]:
%%bash 
cd etape8_nvshmem
make all

In [ ]:
%%bash 
cd etape8_nvshmem
nv-nsight-cu-cli --csv --report-file rapport_etape8_nvshmem.csv ./main
cat rapport_etape8_nvshmem.csv

## etape9_nvshmem_lt
**Description :** NVSHMEM + LTO : ajout de l’optimisation link-time pour inliner les fonctions critiques et réduire le coût des appels.

**Intérêt :** Optimisation link-time pour inliner les sections critiques.

In [ ]:
%%bash cd etape9_nvshmem_lt
make all

In [ ]:
%%bash 
cd etape9_nvshmem_lt
nv-nsight-cu-cli --csv --report-file rapport_etape9_nvshmem_lt.csv ./main
cat rapport_etape9_nvshmem_lt.csv

## etape10_vshmem_neighborhood_lto
**Description :** vshmem neighborhood_sync + LTO : synchronisation fine-grain de voisinage et optimisations link-time O2.

**Intérêt :** Synchronisation fine et LTO pour boucles serrées.

In [ ]:
%%bash 
cd etape10_vshmem_neighborhood_lto
make all

In [ ]:
%%bash 
cd etape10_vshmem_neighborhood_lto
nv-nsight-cu-cli --csv --report-file rapport_etape10_vshmem_neighborhood_lto.csv ./main 
cat rapport_etape10_vshmem_neighborhood_lto.csv

## etape11_nvshmem_norm_overlap_neighborhood_sync_lto
**Description :** Combinaison : NVSHMEM avec recouvrement, synchrone de voisinage, et LTO pour maximiser la concurrence.

**Intérêt :** Combinaison des meilleures pratiques pour un binaire ultra-optimisé.

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto
make all

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto
nv-nsight-cu-cli --csv --report-file rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv ./main
cat rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv

## etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1
**Description :** Tuning étendu : paramètres ajustables (taille de tuile, ordre de boucles) et hooks de benchmark.

**Intérêt :** Ajout de paramètres de tuning et hooks de benchmarking.

In [ ]:
%%bash 
cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 
make all

In [ ]:
%%bash 
cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 
nv-nsight-cu-cli --csv --report-file rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv ./main
cat rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv